In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import numpy as np
from RRNN import RRNN
import matplotlib.pyplot as plt

In [2]:
net = RRNN(ring=True, recurrent=True)

In [ ]:
net.sim_params['b_input']

In [ ]:
bw_values = 180*np.logspace(0, -4, 5, base=2)
fig, axs = plt.subplots(1, 2, figsize=(8, 4))

for i in range(2):
    for bw_value in bw_values:
        if i==0:
            net = RRNN(ring=True, recurrent=False)
        else:
            net = RRNN(ring=True, recurrent=True)

        net.sim_params['b_input'] = bw_value
        df, spikesE, spikesI = net.model()
        theta, fr, result = net.fit_vonMises(spikesE)
        #print(result.best_fit.mean())
        axs[i].plot(theta*180/np.pi, result.best_fit, label=str(bw_value))

    axs[i].set_xlabel('orientation')
    axs[i].set_ylabel('firing rate')
    axs[i].axis('tight')
    axs[i].set_ylim([0, 30])
axs[0].set_title('feed-forward ring')
axs[1].set_title('recurrent ring')

plt.legend(loc='best')
plt.show()

fig.savefig('../figs/ring_feed-forward_vs_recurrent.png', dpi = 600)

In [ ]:
bw_values = 180*np.logspace(-1, -10, 15, base=2)

def HWHH(k):
    """
    See http://motionclouds.invibe.net/posts/testing-grating.html#tuning-the-bandwidth
    
    """
    return .5*np.arccos(1+ np.log((1+np.exp(-2*k))/2)/k)

HWHH_in = HWHH(1/np.sqrt(bw_values/180*np.pi))

print(bw_values, HWHH_in)

In [ ]:
BW= np.zeros((2, len(bw_values)))
for i in range(2):
    for i_bw, bw_value in enumerate(bw_values):
        if i==0:
            net = RRNN(ring=True, recurrent=False)
        else:
            net = RRNN(ring=True, recurrent=True)

        net.sim_params['b_input'] = bw_value
        
        df, spikesE, spikesI = net.model()
        theta, fr, result = net.fit_vonMises(spikesE)
        BW[i, i_bw] = result.params['sigma'].value

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.plot(HWHH_in, HWHH(1/np.sqrt(BW[0, :])), 'k.', label='feed-forward ring')
ax.plot(HWHH_in, HWHH(1/np.sqrt(BW[1, :])), 'r.', label='recurrent ring')
        
ax.set_xlabel('HWHH in')
ax.set_ylabel('HWHH out')

plt.legend(loc='best')
ax.axis('tight')
ax.set_xlim([0, .7])
ax.set_ylim([0, .7])
plt.show()

fig.savefig('../figs/ring_feed-forward_vs_recurrent_BW.png', dpi = 600)